# Chroma With Langchain

- Author: [Gwangwon Jung](https://github.com/pupba)
- Design: []()
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/09-VectorStore/02-Chroma.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/09-VectorStore/02-Chroma.ipynb)

## Overview

This tutorial covers how to use `Chroma Vector Store` with `LangChain` .

`Chroma` is an `open-source AI application database` .

In this tutorial, after learning how to use `langchain-chroma` , we will implement examples of a simple **Text Search** engine using `Chroma` .

![search-example](./assets/02-chroma-with-langchain-flow-search-example.png)

### Table of Contents

- [Overview](#overview)
- [Environement Setup](#environment-setup)
- [What is Chroma?](#what-is-chroma?)
- [LangChain Chroma Basic](#langchain-chroma-basic)
- [Manage Store](#manage-store)
- [Query vector store](#query-vector-store)


### References

- [Chroma Docs](https://docs.trychroma.com/docs/overview/introduction)
- [Langchain-Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
- [List of VectorStore supported by Langchain](https://python.langchain.com/docs/integrations/vectorstores/)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [166]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [432]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain-core",
        "langchain-chroma",
        "chromadb",
        "langchain-text-splitters",
        "langchain-huggingface",
        "python-dotenv",
    ],
    verbose=False,
    upgrade=False,
)

In [168]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Chroma",
    }
)

Environment variables have been set successfully.


You can alternatively set API keys such as `OPENAI_API_KEY` in a `.env` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [169]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

## What is Chroma?

![logo](./assets/02-chroma-with-langchain-chroma-logo.png)

`Chroma` is the `open-source vector database` designed for AI application. 

It specializes in storing high-dimensional vectors and performing fast similariy search, making it ideal for tasks like `semantic search` , `recommendation systems` and `multimodal search` .

With its **developer-friendly APIs** and seamless integration with frameworks like `LangChain` , `Chroma` is powerful tool for building scalable, AI-driven solutions.

The biggest feature of `Chroma` is that it internally **Indexing ([HNSW](https://en.wikipedia.org/wiki/Hierarchical_navigable_small_world))** and **Embedding ([all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2))** are used when storing data.

## LangChain Chroma Basic

### Create VectorDB

The **library** supported by `LangChain` has no `upsert` function and lacks interface uniformity with other **Vector DBs**, so we have implemented a new **Python** class.

First, Defines a **Python** class.

In [445]:
from utils.vectordbinterface import VectorDBInterface
from langchain_chroma import Chroma
import chromadb
from chromadb.utils import embedding_functions
from langchain_core.documents import Document
from typing import List, Dict, Any, Callable, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed
from uuid import uuid4


class ChromaDB(VectorDBInterface):
    def __init__(self, embeddings: Optional[Any] = None):
        self.chroma = None
        self.unique_ids = set()
        self._embeddings = embeddings if embeddings is not None else None
        self._embeddings_function = (
            embeddings.embed_documents
            if embeddings is not None
            else embedding_functions.DefaultEmbeddingFunction()  # all-MiniLM-L6v2
        )
        self.chroma_search = None

    def connect(self, **kwargs) -> None:
        """
        ChromaDB Connect
        """
        langchain_config = {}

        if kwargs["mode"] == "in-memory":  # In-Memory
            chroma_client = chromadb.Client()

        elif kwargs["mode"] == "persistent":  # Local
            chroma_client = chromadb.PersistentClient(path=kwargs["persistent_path"])
            langchain_config["persist_directory"] = kwargs["persistent_path"]

        elif kwargs["mode"] == "server":  # Server-Client
            chroma_client = chromadb.HttpClient(
                host=kwargs["host"], port=kwargs["port"]
            )
        else:
            raise Exception(
                "Invalid Input, Enter one of ['in-meory','persistent','server'] modes."
            )

        # The Chroma client allows you to get and delete existing collections by their name.
        # It also offers a get or create method to get a collection if it exists, or create it otherwise.
        self.chroma = chroma_client.get_or_create_collection(name=kwargs["collection"])
        langchain_config["collection_name"] = kwargs["collection"]

        existing_ids = self.chroma.get(include=[])["ids"]  # Get existing unique ids
        self.unique_ids.update(existing_ids)  # current unique ids update

        # Langchain-chroma for Search
        self.chroma_search = Chroma(
            **langchain_config,
            embedding_function=self._embeddings,
        )

    def create_index(
        self, index_name: str, dimension: int, metric: str = "dotproduct", **kwargs
    ) -> Any:
        """
        Not used in Chroma
        """
        return None

    def get_index(self, index_name: str) -> Any:
        """
        Not used in Chroma
        """
        return None

    def delete_index(self, index_name: str) -> None:
        """
        Not used in Chroma
        """
        return None

    def list_indexs(self) -> List[str]:
        """
        Not used in Chroma
        """
        return None

    def add(self, pre_documents: List[Document], **kwargs) -> None:
        documents = []
        metadatas = []
        ids = []
        for doc in pre_documents:
            documents.append(doc.page_content)
            ids.append(doc.metadata["id"])
            metadatas.append(
                {key: value for key, value in doc.metadata.items() if key != "id"}
            )

        embeddings = self._embeddings_function(documents)  # embedding documents

        self.chroma.add(
            documents=documents,
            embeddings=embeddings,
            metadatas=metadatas,
            ids=ids,
        )
        self.unique_ids.update(ids)

    def upsert_documents(
        self,
        documents: List[Dict],
        **kwargs,
    ) -> None:
        """
        Upsert documents to Chroma

        :param documents: List of documents
        :param embedding_function: Embedding function
        """
        # Embedding documents
        embeddings = self._embeddings_function([doc.page_content for doc in documents])
        # Generate unique ids
        unique_ids = [doc.metadata["id"] for doc in documents]
        # Upsert documents
        self.chroma.upsert(
            ids=unique_ids,
            embeddings=embeddings,
            metadatas=[doc.metadata for doc in documents],
            documents=[doc.page_content for doc in documents],
        )

        print("Success Upsert All Documents")

        # update unique_ids
        self.unique_ids.update(unique_ids)

    def upsert_documents_parallel(
        self,
        documents: List[Dict],
        batch_size: int = 32,
        max_workers: int = 10,
        **kwargs,
    ) -> None:
        """
        Parallel upsert documents to Chroma
        :param documents: List of documents
        :param batch_size: Batch size
        :param max_workers: Number of workers
        """
        # split documents into batches
        batches = [
            documents[i : i + batch_size] for i in range(0, len(documents), batch_size)
        ]
        all_unique_ids = set()  # Store all unique IDs from all batches
        failed_uids = []  # Store failed batches

        # Parallel processing
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [
                executor.submit(self.upsert_documents, batch, **kwargs)
                for batch in batches
            ]

        # Wait for all futures to complete
        for future, batch in zip(as_completed(futures), batches):
            try:
                future.result()  # Wait for the batch to complete
                # Extract unique IDs from the batch
                unique_ids = [doc.metadata["id"] for doc in batch]
                all_unique_ids.update(unique_ids)  # Add to the total set
            except Exception as e:
                print(f"An error occurred during upsert: {e}")
                failed_uids.append(unique_ids)  # Store failed batch for retry

        self.unique_ids.update(all_unique_ids)

        print(f"Success Upsert Parallel All Documents\nFailed Batches: {failed_uids}")

    def query(
        self, query_vector: List[float], top_k: int = 10, **kwargs
    ) -> List[Document]:
        """
        Search in LangChain is better and uses its functionality
        """
        pass

    def delete_by_filter(
        self, unique_ids: List[str], filters: Optional[Dict] = None, **kwargs
    ) -> None:
        """
        Delete documents by filter
        :param unique_ids: List of unique ids
        :param filters: Filter conditions
        """
        try:
            self.chroma.delete(
                ids=unique_ids,
                where=filters,
            )
            pre_count = len(self.unique_ids)
            self.unique_ids = set(self.chroma.get(include=[])["ids"])

            print(f"Success Delete {pre_count-len(self.unique_ids)} Documents")

        except Exception as e:
            print(f"Error: {e}")

    def preprocess_documents(
        self,
        documents: List[Document],
        source: Optional[str] = None,
        author: Optional[str] = None,
        chapter: bool = False,
        **kwargs,
    ) -> List[Dict]:
        """
        Change LangChain Document to Chroma
        :param documents: List of LangChain documents
        :param source: Source of the document
        :param author: Author of the document
        :param chapter: Chapter of the document
        :return: List of Chroma documents
        """
        metadata = {}

        if source is not None:
            metadata["source"] = source
        if author is not None:
            metadata["author"] = author

        processed_docs = []
        current_chapter = None
        save_flag = False
        for doc in documents:
            content = doc.page_content

            content = content.replace("(picture)\n", "")

            # Chapter dectect
            if content.startswith("[ Chapter ") and "\n" in content:
                # Chapter Num (example: "[ Chapter 26 ]\n" -> 26)
                chapter_part, content_part = content.split("\n", 1)
                current_chapter = int(chapter_part.split()[2].strip("]"))
                content = content_part

            elif content.strip() == "[ END ]":
                break

            if current_chapter is not None:
                # add metadata
                if chapter:
                    metadata["chapter"] = current_chapter
                updated_metadata = {**doc.metadata, **metadata, "id": str(uuid4())}
                # Document append to processed_docs
                processed_docs.append(
                    Document(metadata=updated_metadata, page_content=content)
                )

        return processed_docs

    def get_api_key(self) -> str:
        """
        Not used in Chroma
        """
        return None

### Select Embedding Model

We load the **Embedding Model** with `langchain_huggingface` .

If you want to use a different model, use a different model.

In [439]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "Alibaba-NLP/gte-base-en-v1.5"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs={"trust_remote_code": True}
)

Create `ChromaDB` object.

In [446]:
vector_store = ChromaDB(embeddings=embeddings)
vector_store.connect(
    mode="persistent",
    persistent_path="data/chroma.sqlite",
    collection="test",
)

### Load Text Documents Data

In this tutorial, we will use the `A Little Prince` fairy tale document.

To put this data in `Chroma` ,we will do data preprocessing first.

First of all, we will load the `data/the_little_prince.txt` file that extracted only the text of the fairy tale document.


In [183]:
# If your "OS" is "Windows", add 'encoding=utf-8' to the open function
with open("./data/the_little_prince.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

Second, chunking the text imported into the `RecursiveCharacterTextSplitter` .

In [246]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

split_docs = text_splitter.create_documents([raw_text])

for docs in split_docs[:3]:
    print(f"Content: {docs.page_content}\nMetadata: {docs.metadata}", end="\n\n")

Content: The Little Prince
Written By Antoine de Saiot-Exupery (1900〜1944)
Metadata: {}

Content: [ Antoine de Saiot-Exupery ]
Metadata: {}

Content: Over the past century, the thrill of flying has inspired some to perform remarkable feats of
Metadata: {}



Preprocessing document for `Chroma` .

In [275]:
pre_dosc = vector_store.preprocess_documents(
    documents=split_docs,
    source="The Little Prince",
    author="Antoine de Saint-Exupéry",
    chapter=True,
)

In [279]:
pre_dosc[:3]

[Document(metadata={'source': 'The Little Prince', 'author': 'Antoine de Saint-Exupéry', 'chapter': 1, 'id': 'dcccc8df-23a7-4250-b258-a1508cb3afa5'}, page_content='- we are introduced to the narrator, a pilot, and his ideas about grown-ups'),
 Document(metadata={'source': 'The Little Prince', 'author': 'Antoine de Saint-Exupéry', 'chapter': 1, 'id': '5cb1faed-692e-4aa0-9cd6-dfdf2fc23dac'}, page_content='Once when I was six years old I saw a magnificent picture in a book, called True Stories from'),
 Document(metadata={'source': 'The Little Prince', 'author': 'Antoine de Saint-Exupéry', 'chapter': 1, 'id': 'edbfea7b-e62f-4563-87a6-a9bf5a3d8b2b'}, page_content='True Stories from Nature, about the primeval forest. It was a picture of a boa constrictor in the')]

## Manage Store

This section introduces three basic functions.

- `add`

- `upsert(parallel)`

- `delete`

### Add

Add the new `Documents` .

An error occurs if you have the same `ID` .

In [281]:
vector_store.add(pre_documents=pre_dosc[:4])

In [299]:
uids = list(vector_store.unique_ids)
uids

['52a8a21b-112a-499b-b8a4-3f2ef902a988',
 'edbfea7b-e62f-4563-87a6-a9bf5a3d8b2b',
 '5cb1faed-692e-4aa0-9cd6-dfdf2fc23dac',
 'dcccc8df-23a7-4250-b258-a1508cb3afa5']

In [283]:
vector_store.chroma.get(ids=uids[0])

{'ids': ['edbfea7b-e62f-4563-87a6-a9bf5a3d8b2b'],
 'embeddings': None,
 'documents': ['True Stories from Nature, about the primeval forest. It was a picture of a boa constrictor in the'],
 'uris': None,
 'data': None,
 'metadatas': [{'author': 'Antoine de Saint-Exupéry',
   'chapter': 1,
   'source': 'The Little Prince'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

Error occurs when trying to `add` duplicate `ids` .

In [284]:
vector_store.add(pre_documents=pre_dosc[:4])

Add of existing embedding ID: dcccc8df-23a7-4250-b258-a1508cb3afa5
Add of existing embedding ID: 5cb1faed-692e-4aa0-9cd6-dfdf2fc23dac
Add of existing embedding ID: edbfea7b-e62f-4563-87a6-a9bf5a3d8b2b
Add of existing embedding ID: 52a8a21b-112a-499b-b8a4-3f2ef902a988
Insert of existing embedding ID: dcccc8df-23a7-4250-b258-a1508cb3afa5
Insert of existing embedding ID: 5cb1faed-692e-4aa0-9cd6-dfdf2fc23dac
Insert of existing embedding ID: edbfea7b-e62f-4563-87a6-a9bf5a3d8b2b
Insert of existing embedding ID: 52a8a21b-112a-499b-b8a4-3f2ef902a988


In [300]:
uids = list(vector_store.unique_ids)
uids

['52a8a21b-112a-499b-b8a4-3f2ef902a988',
 'edbfea7b-e62f-4563-87a6-a9bf5a3d8b2b',
 '5cb1faed-692e-4aa0-9cd6-dfdf2fc23dac',
 'dcccc8df-23a7-4250-b258-a1508cb3afa5']

### Upsert(parallel)

`Upsert` will `Update` a document or `Add` a new document if the same `ID` exists.

In [306]:
tmp_ids = [docs.metadata["id"] for docs in pre_dosc[:2]]
vector_store.chroma.get(ids=tmp_ids)

{'ids': ['dcccc8df-23a7-4250-b258-a1508cb3afa5',
  '5cb1faed-692e-4aa0-9cd6-dfdf2fc23dac'],
 'embeddings': None,
 'documents': ['- we are introduced to the narrator, a pilot, and his ideas about grown-ups',
  'Once when I was six years old I saw a magnificent picture in a book, called True Stories from'],
 'uris': None,
 'data': None,
 'metadatas': [{'author': 'Antoine de Saint-Exupéry',
   'chapter': 1,
   'id': 'dcccc8df-23a7-4250-b258-a1508cb3afa5',
   'source': 'The Little Prince'},
  {'author': 'Antoine de Saint-Exupéry',
   'chapter': 1,
   'id': '5cb1faed-692e-4aa0-9cd6-dfdf2fc23dac',
   'source': 'The Little Prince'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [308]:
pre_dosc[0].page_content = "Changed Content"
pre_dosc[0]

Document(metadata={'source': 'The Little Prince', 'author': 'Antoine de Saint-Exupéry', 'chapter': 1, 'id': 'dcccc8df-23a7-4250-b258-a1508cb3afa5'}, page_content='Changed Content')

In [310]:
vector_store.upsert_documents(
    documents=pre_dosc[:2],
)
tmp_ids = [docs.metadata["id"] for docs in pre_dosc[:2]]
vector_store.chroma.get(ids=tmp_ids)

Success Upsert All Documents


{'ids': ['dcccc8df-23a7-4250-b258-a1508cb3afa5',
  '5cb1faed-692e-4aa0-9cd6-dfdf2fc23dac'],
 'embeddings': None,
 'documents': ['Changed Content',
  'Once when I was six years old I saw a magnificent picture in a book, called True Stories from'],
 'uris': None,
 'data': None,
 'metadatas': [{'author': 'Antoine de Saint-Exupéry',
   'chapter': 1,
   'id': 'dcccc8df-23a7-4250-b258-a1508cb3afa5',
   'source': 'The Little Prince'},
  {'author': 'Antoine de Saint-Exupéry',
   'chapter': 1,
   'id': '5cb1faed-692e-4aa0-9cd6-dfdf2fc23dac',
   'source': 'The Little Prince'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [ ]:
# parallel upsert
vector_store.upsert_documents_parallel(
    documents=pre_dosc,
    batch_size=32,
    max_workers=10,
)
# Clear Output cell, because it is too long.

In [354]:
len(vector_store.unique_ids)

1317

### Delete

`Delete` the Documents.

You can use with `filter` .

In [355]:
len([docs for docs in pre_dosc if docs.metadata["chapter"] == 1])

43

In [356]:
vector_store.delete_by_filter(
    unique_ids=list(vector_store.unique_ids), filters={"chapter": 1}
)

Success Delete 43 Documents


In [357]:
len(vector_store.unique_ids)

1274

In [358]:
vector_store.delete_by_filter(unique_ids=list(vector_store.unique_ids)[:30])

Success Delete 30 Documents


In [362]:
len(vector_store.unique_ids)

1244

## Query Vector Store

There are two ways to `Query` the `Vector Store` .

- **Directly** : Query the vector store directly using methods like `similarity_search` or `similarity_search_with_score` .

- **Turning into retriever** : Convert the vector store into a `retriever` object, which can be used in `LangChain` pipelines or chains.

### similarity_search()

`similarity_search()` is run similarity search with Chroma.

**Parameters**

- `query:str` - Query text to search for.

- `k: int = DEFAULT_K` - Number of results to return. Defaults to 4.    

- `filter: Dict[str, str] | None = None` - Filter by metadata. Defaults to None.

- `**kwargs:Any` - Additional keyword arguments to pass to Chroma collection query.


**Returns**
- `List[Documents]` - List of documents most similar to the query text.



### similarity_search_with_score()

`similarity_search_with_score()` is run similarity search with Chroma with distance.

**Parameters**

- `query:str` - Query text to search for.

- `k:int = DEFAULT_K` - Number of results to return. Defaults to 4.

- `filter: Dict[str, str] | None = None` - Filter by metadata. Defaults to None.

- `where_document: Dict[str, str] | None = None` - dict used to filter by the documents. E.g. {$contains: {"text": "hello"}}.

- `**kwargs:Any` : Additional keyword arguments to pass to Chroma collection query.


**Returns**
- `List[Tuple[Document, float]]` - List of documents most similar to the query text and distance in float for each. Lower score represents more similarity.

In [455]:
# Directly - similarity_search
results = vector_store.chroma_search.similarity_search(
    query="Look at it",
    k=2,
    filter={"chapter": 20},
)

for idx, res in enumerate(results):
    print(f"{idx}: {res.page_content}")

0: - the little prince discovers a garden of roses
1: one of her kind in all the universe. And here were five thousand of them, all alike, in one single


In [461]:
# Directly - similarity_search_with_score

results = vector_store.chroma_search.similarity_search_with_score(
    query="Look at it",
    k=1,
    filter={"chapter": 10},
)

for idx, (res, score) in enumerate(results):
    print(f"{idx}: [Similarity Score: {round(score,3)*100}%] {res.page_content}")

0: [Similarity Score: 46623.700000000004%] "Oh, but I have looked already!" said the little prince, turning around to give one more glance to


### as_retriever()

The `as_retriever()` method converts a `VectorStore` object into a `Retriever` object.

A `Retriever` is an interface used in `LangChain` to query a vector store and retrieve relevant documents.

**Parameters**

- `search_type:Optional[str]` - Defines the type of search that the Retriever should perform. Can be `similarity` (default), `mmr` , or `similarity_score_threshold`

- `search_kwargs:Optional[Dict]` - Keyword arguments to pass to the search function. 

    Can include things like:

    `k` : Amount of documents to return (Default: 4)

    `score_threshold` : Minimum relevance threshold for similarity_score_threshold

    `fetch_k` : Amount of documents to pass to `MMR` algorithm(Default: 20)
        
    `lambda_mult` : Diversity of results returned by MMR; 1 for minimum diversity and 0 for maximum. (Default: 0.5)

    `filter` : Filter by document metadata


**Returns**

- `VectorStoreRetriever` - Retriever class for VectorStore.


### invoke()

Invoke the retriever to get relevant documents.

Main entry point for synchronous retriever invocations.

**Parameters**

- `input:str` - The query string.
- `config:RunnableConfig | None = None` - Configuration for the retriever. Defaults to None.
- `**kwargs:Any` - Additional arguments to pass to the retriever.


**Returns**

- `List[Document]` : List of relevant documents.

In [463]:
retriever = vector_store.chroma_search.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

retriever.invoke("a little prince")

[Document(id='98325aa3-653c-4390-abe6-a4ade76f792f', metadata={'author': 'Antoine de Saint-Exupéry', 'chapter': 10, 'id': '98325aa3-653c-4390-abe6-a4ade76f792f', 'source': 'The Little Prince'}, page_content='- the little prince visits the king'),
 Document(id='2722874b-928d-4830-a34a-4f0a375794f3', metadata={'author': 'Antoine de Saint-Exupéry', 'chapter': 7, 'id': '2722874b-928d-4830-a34a-4f0a375794f3', 'source': 'The Little Prince'}, page_content='- the narrator learns about the secret of the little prince‘s life')]

Remove a `Huggingface Cache`

In [466]:
from huggingface_hub import scan_cache_dir

del embeddings
scan = scan_cache_dir()
scan.delete_revisions()

DeleteCacheStrategy(expected_freed_size=0, blobs=frozenset(), refs=frozenset(), repos=frozenset(), snapshots=frozenset())